<a href="https://colab.research.google.com/github/nananatsu/blog/blob/master/PQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

准备测试数据

In [ ]:
from google.colab import drive

drive.mount('/content/drive/')

!mkdir -p mydata
!unzip /content/drive/MyDrive/train_data/baike2018qa.zip -d mydata

Mounted at /content/drive/
Archive:  /content/drive/MyDrive/train_data/baike2018qa.zip
  inflating: mydata/baike_qa_train.json  
  inflating: mydata/baike_qa_valid.json  


In [ ]:
%pip install numpy pandas scipy sentence-transformers torch faiss-gpu

In [ ]:
import pandas as pd

data = pd.read_json('./mydata/baike_qa_train.json', lines=True, nrows=5000)

sentences = data['title']

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('DMetaSoul/sbert-chinese-general-v2')
sentence_embeddings = model.encode(sentences)

sentence_embeddings.shape

(5000, 768)

量化是将输入值从大集合（通常是连续的）映射到较小集合（通常具有有限数量元素），舍入（用更短、更简单、更明确的近似值进行替换）和截断（限制小数点右边的位数）是典型的量化过程，量化构成了所有有损压缩算法的核心。输入值与量化值间的差异称为量化误差，执行量化的设备或算法功能称为量化器。

PQ（乘积量化）通过将向量空间分解为低位子空间的笛卡尔积，每个子空间独自进行量化，这样一个向量能够被子空间量化后的索引组成的短编码表示，两个向量间的L2距离可以通过量化后的编码进行估计。


定义$q$是一个量化函数，将D维向量$x \in \mathbb{R}^D$映射为向量$q(x) \in C $，$C = \{c_i;i \in I \}$是一个大小为k的编码簿， 其中$I = 0 ... k-1$是一个有限的索引集合，$c_i$被称为质心。

将所有映射到同一索引$i$的向量划分为一个单元(Voronoi cell) $V_i$：

$V_i \mathop{=}\limits^{Δ} \{x ∈ \mathbb{R}^D : q(x) = c_i \}$，

量化器的$k$个单元是向量空间$\mathbb{R}^D$的一个分区，在同一个单元$V_i$中向量可以使用同一个质心$c_i$进行重构，量化器的好坏可以用输入向量与其再现值$q(x)$间的均方误差(MSE)来度量，使用$d(x,y) = \Vert x -y \Vert$表示两个向量间的L2距离，$p(x)$表示随机变量$X$的概率分布函数，则其均方误差为：

$MSE(q) = \mathbb{E}_X[d(q(x),x)^2] = ∫ p(x) d(q(x),x)^2 dx$


In [ ]:
x = sentence_embeddings[0]

D = len(x)
k = 2**5
c = []
for i in range(k):
  c_i =  [randint(0, 9) for _ in range(D)]
  c.append(c_i)

In [ ]:
from scipy.spatial import distance

min_distance = 9e9
idx = -1
for i in range(k):
  l2_distance = distance.euclidean(x,c[i])
  if l2_distance < min_distance:
    idx = i
    min_distance = l2_distance
idx

5

In [ ]:
import numpy as np

mse = (np.square(x - c[idx])).mean()

mse

27.457359910837244

为使量化器最优，需要满足劳埃德(Lloyd)最优条件：
- 向量$x$需要被量化到最近的质心，以L2距离作为距离函数则：$q(x) = arg \: \mathop{min}\limits_{c_i \in C} d(x,c_i)$，这样单元间被超平面划分。
- 质心必须是Voronoi单元中的向量的期望：$c_i = \mathbb{E}_X[x|i] = \int_{V_i} p(x) x dx$，劳埃德量化器迭代分配向量到质心并从分配后的向量集合中重新估计质心。

当从质心重建单元$V_i$中的向量时，使用均方失真来评估量化器好坏，$p_i = \mathbb{P}(q(x) = c_i)$表示向量$x$被分配到质心$c_i$的概率，则均方失真为：

$ξ(q,c_i) = \frac{1}{p_i}\int_{V_i}d(x,q(x))^2 p(x) dx$

均方误差可以使用均方失真得出：

$MSE(q) = \mathop{\sum}\limits_{i \in I} p_i ξ(q,c_i) $



假设要将一个128维的向量将其量化为64位的编码，需要$k=2^{64}$个质心，量化不改变向量维数，则需要存储$D × k = 2^{64} \times 128$个浮点值来表示质心，这使得在内存进行向量量化存在困难。乘积量化是通过允许选择进行联合量化的组件数量来解决存储及复杂性问题。

将输入向量$x$切分为$m$个不同的子向量$u_j, 1 ≤ j ≤ m$，子向量维度$D^* = D/m$，D是m的倍数，将子向量用m个不同量化器分别进行量化，$q_j$是第j个子向量使用的量化器，通过子量化器$q_j$关联索引集合$I_j$，对应到编码簿$C_j$及相应的质心$c_{j,i}$

$ \underbrace{x_1,...,x_{D^*},}_{u_1(x)} ...,\underbrace{x_{D-D^*+1},...,x_D}_{u_m(x)} → q_1(u_1(x)),...,q_m(u_m(x))$

乘积量器再现值由索引集合的笛卡尔积$I = I_1 × ... × I_m $确定，相应的编码簿为$C = C_1 × ... × C_m$，集合中的元素对应向量经m个子量化器处理后的质心，假设所有的子量化器有着有限个数$k^*$个再现值，总质心数$k = (k^*)^m$

In [ ]:
x = [1, 8, 3, 9, 1, 2, 9, 4, 5, 4, 6, 2]

m = 4
D = len(x)

assert D % m == 0

D_ = int(D / m)

u = [x[row:row+D_] for row in range(0, D, D_)]
u

[[1, 8, 3], [9, 1, 2], [9, 4, 5], [4, 6, 2]]

In [ ]:
k = 2**5
assert k % m == 0
k_ = int(k/m)
print(f"{k=}, {k_=}")

k=32, k_=8


In [ ]:
from random import randint

c = []  # 编码簿，存储所有可能质心
for j in range(m):
    # j 表示子向量（量化器）位置
    c_j = []
    for i in range(k_):
        # i 表示子向量空间中聚类/再现值位置
        c_ji = [randint(0, 9) for _ in range(D_)]
        c_j.append(c_ji)  # 添加聚类质心到子编码簿
    # 将子编码簿添加到编码簿
    c.append(c_j)
c

[[[2, 1, 1],
  [1, 0, 5],
  [7, 7, 1],
  [3, 2, 8],
  [5, 0, 8],
  [7, 7, 0],
  [1, 0, 2],
  [5, 7, 5]],
 [[1, 8, 3],
  [8, 9, 4],
  [4, 4, 1],
  [5, 6, 0],
  [2, 8, 3],
  [8, 2, 5],
  [3, 1, 0],
  [0, 2, 7]],
 [[0, 9, 7],
  [6, 4, 1],
  [8, 3, 8],
  [8, 1, 6],
  [0, 6, 5],
  [7, 8, 6],
  [9, 9, 1],
  [0, 0, 5]],
 [[4, 8, 9],
  [1, 9, 3],
  [8, 6, 8],
  [2, 8, 6],
  [3, 6, 9],
  [2, 1, 1],
  [2, 2, 5],
  [1, 5, 6]]]

In [ ]:
def euclidean(v, u):
    distance = sum((x - y) ** 2 for x, y in zip(v, u)) ** .5
    return distance

def nearest(c_j, u_j):
    distance = 9e9
    for i in range(k_):
        new_dist = euclidean(c_j[i], u_j)
        if new_dist < distance:
            nearest_idx = i
            distance = new_dist
    return nearest_idx

In [ ]:
ids = []
for j in range(m):
    i = nearest(c[j], u[j])
    ids.append(i)
ids

[7, 5, 2, 1]

In [ ]:
q = []
for j in range(m):
    c_ji = c[j][ids[j]]
    q.extend(c_ji)

In [ ]:
q

[5, 7, 5, 8, 2, 5, 8, 3, 8, 1, 9, 3]

In [ ]:
def mse(v, u):
    error = sum((x - y) ** 2 for x, y in zip(v, u)) / len(v)
    return error

In [ ]:
mse(x, q)

5.166666666666667

乘积量化的优势在于通过几个小的质心集合生成一个大的质心集合，只需要存储子量化器对应的$m \times k^*$个质心，总计$mk^*D^*$个浮点值，即相应内存使用及复杂性为$mk^*D^* = k^{(1/m)}D $，相较其他量化方法k-means、HKM，PQ能够在内存中对较大k值得向量进行索引。

在向量中连续的元素在结构上通常是相关的，最好使用同一个子量化器进行量化，由于子向量空间是正交的，量化器的均方失真可以表示为：

$MSE(q) = \mathop{\sum}\limits_{j} MSE(q_j) $

更高的$k^*$会造成更高的计算复杂度，更大的内存占用，通常$k^* = 256,m = 8$是一个合理的选择。

有两种方法来计算查询向量与量化后向量的距离
- 对称距离计算(SDC)：将查询向量也进行量化，计算量化后向量间的距离
- 非对称距离计算(ADC)：不对查询向量量化，直接计算距离